### Thư viện sử dụng

In [32]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import math

In [33]:
dataset_ratings=pd.read_csv("ratings_Beauty.csv")

Xem số dòng và cột

In [34]:
dataset_ratings.shape

(2023070, 4)

Số dòng dữ liệu cho từng cột

In [35]:
dataset_ratings.count()

UserId       2023070
ProductId    2023070
Rating       2023070
Timestamp    2023070
dtype: int64

Trình bày tập dữ liệu

In [36]:
dataset_ratings.head

<bound method NDFrame.head of                  UserId   ProductId  Rating   Timestamp
0        A39HTATAQ9V7YF  0205616461     5.0  1369699200
1        A3JM6GV9MNOF9X  0558925278     3.0  1355443200
2        A1Z513UWSAAO0F  0558925278     5.0  1404691200
3        A1WMRR494NWEWV  0733001998     4.0  1382572800
4        A3IAAVS479H7M7  0737104473     1.0  1274227200
...                 ...         ...     ...         ...
2023065  A3DEHKPFANB8VA  B00LORWRJA     5.0  1405296000
2023066  A3DEHKPFANB8VA  B00LOS7MEE     5.0  1405296000
2023067   AG9TJLJUN5OM3  B00LP2YB8E     5.0  1405382400
2023068   AYBIB14QOI9PC  B00LPVG6V0     5.0  1405555200
2023069  A1PB6OT0OOPKNQ  B00LU0LTOU     5.0  1405641600

[2023070 rows x 4 columns]>

Trình bày 5 dòng dữ liệu đầu tiên

In [37]:
dataset_ratings.head(5)

,UserId,ProductId,Rating,Timestamp
0,A39HTATAQ9V7YF,0205616461,5.0,1369699200
1,A3JM6GV9MNOF9X,0558925278,3.0,1355443200
2,A1Z513UWSAAO0F,0558925278,5.0,1404691200
3,A1WMRR494NWEWV,0733001998,4.0,1382572800
4,A3IAAVS479H7M7,0737104473,1.0,1274227200


Trình bày 5 dòng dữ liệu cuối cùng

In [38]:
dataset_ratings.tail(5)

,UserId,ProductId,Rating,Timestamp
2023065,A3DEHKPFANB8VA,B00LORWRJA,5.0,1405296000
2023066,A3DEHKPFANB8VA,B00LOS7MEE,5.0,1405296000
2023067,AG9TJLJUN5OM3,B00LP2YB8E,5.0,1405382400
2023068,AYBIB14QOI9PC,B00LPVG6V0,5.0,1405555200
2023069,A1PB6OT0OOPKNQ,B00LU0LTOU,5.0,1405641600


### Kiểm dữ liệu NULL và NA trong dataset

In [39]:
dataset_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2023070 entries, 0 to 2023069
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   UserId     object 
 1   ProductId  object 
 2   Rating     float64
 3   Timestamp  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 61.7+ MB


In [40]:
dataset_ratings.isnull().any()

UserId       False
ProductId    False
Rating       False
Timestamp    False
dtype: bool

### Xóa dữ liệu NULL

In [41]:
new_dataset_rating = dataset_ratings.dropna()

Dữ liệu còn lại sau khi xóa NULL

In [42]:
new_dataset_rating.count()

UserId       2023070
ProductId    2023070
Rating       2023070
Timestamp    2023070
dtype: int64

In [43]:
new_dataset_rating.shape

(2023070, 4)

In [44]:
#Kiểm tra các kiểu dữ liệu
new_dataset_rating.dtypes

UserId        object
ProductId     object
Rating       float64
Timestamp      int64
dtype: object

In [45]:
#Kiểm tra thống kê mô tả
new_dataset_rating.describe().T

,count,mean,std,min,25%,50%,75%,max
Rating,2023070.0,4.149036e+00,1.311505e+00,1.0,4.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00
Timestamp,2023070.0,1.360389e+09,4.611860e+07,908755200.0,1.350259e+09,1.372810e+09,1.391472e+09,1.406074e+09


In [46]:
#Xem thống kê mô tả cột rating
new_dataset_rating.Rating.describe().T

count    2.023070e+06
mean     4.149036e+00
std      1.311505e+00
min      1.000000e+00
25%      4.000000e+00
50%      5.000000e+00
75%      5.000000e+00
max      5.000000e+00
Name: Rating, dtype: float64

Khởi tạo SPARK

In [58]:
import findspark
findspark.init()

import pyspark
findspark.find()

from pyspark.sql import SparkSession
from pyspark.sql.functions import count

spark = (SparkSession
         .builder
         .appName("ALS recommendation spark session")
         .getOrCreate())

Đọc dữ liệu

In [59]:
ratingData = (spark.read
          .option("HEADER", True)
          .option("inferSchema", True)
          .csv("ratings_Beauty.csv")
         )

ratingData.show(5)

+--------------+----------+------+----------+
|        UserId| ProductId|Rating| Timestamp|
+--------------+----------+------+----------+
|A39HTATAQ9V7YF|0205616461|   5.0|1369699200|
|A3JM6GV9MNOF9X|0558925278|   3.0|1355443200|
|A1Z513UWSAAO0F|0558925278|   5.0|1404691200|
|A1WMRR494NWEWV|0733001998|   4.0|1382572800|
|A3IAAVS479H7M7|0737104473|   1.0|1274227200|
+--------------+----------+------+----------+
only showing top 5 rows



Chuyển cột UserId (kiểu string) thành label1 (kiểu double)

In [84]:
from pyspark.ml.feature import StringIndexer

userId_indexer = StringIndexer(inputCol = 'UserId', outputCol = 'label1')

ratingindexed1 = userId_indexer.fit(ratingData).transform(ratingData)

ratingindexed1.show(5)

+--------------+----------+------+----------+--------+
|        UserId| ProductId|Rating| Timestamp|  label1|
+--------------+----------+------+----------+--------+
|A39HTATAQ9V7YF|0205616461|   5.0|1369699200| 70392.0|
|A3JM6GV9MNOF9X|0558925278|   3.0|1355443200|265306.0|
|A1Z513UWSAAO0F|0558925278|   5.0|1404691200|552933.0|
|A1WMRR494NWEWV|0733001998|   4.0|1382572800|536779.0|
|A3IAAVS479H7M7|0737104473|   1.0|1274227200| 14679.0|
+--------------+----------+------+----------+--------+
only showing top 5 rows



Chuyển cột productId (kiểu string) thành label2 (kiểu double)

In [86]:
from pyspark.ml.feature import StringIndexer

productId_indexer = StringIndexer(inputCol = 'ProductId', outputCol = 'label2')

ratingindexed2 = productId_indexer.fit(ratingindexed1).transform(ratingindexed1)

ratingindexed2.show(5)

ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

Khởi tạo sparkDataFrame từ pandas dataFrame

In [81]:
(train, test) = ratingData.randomSplit([.8, .2], seed = 1)

 Xây dựng và đánh giá mô hình ALS trên pyspark.

In [1]:
# Xây dựng mô hình recommendation sử dụng thuật toán ALS trên tập dữ liệu huấn luyện

from datetime import datetime
from pyspark.ml.recommendation import ALS
start_time = datetime.now()
als = ALS(maxIter=5, regParam=0.01, userCol="UserId", itemCol="ProductId", ratingCol="Rating")
model = als.fit(train)
end_time = datetime.now()

print('Execute time {}'.format(end_time - start_time))

AssertionError: 